In [1]:
import numpy as np
import pandas as pd

from sqlalchemy import create_engine

In [2]:
path = '../data/raw/The-Database-of-Political-Institutions-2020-DPI2020/'
institutions = pd.read_stata(path+'DPI2020_stata13.dta')

In [3]:
institutions

,countryname,ifs,year,system,yrsoffc,finittrm,yrcurnt,termlimit,reelect,multpl,...,checks,stabs_strict,stabs,stabns_strict,stabns,tenlong_strict,tenlong,tenshort_strict,tenshort,polariz
0,Turk Cyprus,0,1975-01-01,-999.0,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,NA,NaN,NaN,NaN,NaN,NA,NA,NA,NA,NA
1,Turk Cyprus,0,1976-01-01,Presidential,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,NA,NA,NA,NA,1.0,1.0,1.0,1.0,0.0
2,Turk Cyprus,0,1977-01-01,Presidential,2.0,1.0,4.0,1.0,1.0,1.0,...,3.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,0.0
3,Turk Cyprus,0,1978-01-01,Presidential,3.0,1.0,3.0,1.0,1.0,1.0,...,3.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,2.0,0.0
4,Turk Cyprus,0,1979-01-01,Presidential,4.0,1.0,2.0,1.0,1.0,1.0,...,3.0,0.0,0.0,0.0,0.0,4.0,4.0,3.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8195,Zimbabwe,ZWE,2016-01-01,Presidential,29.0,1.0,2.0,3.0,1.0,1.0,...,4.0,0.0,0.0,0.0,0.0,29.0,29.0,3.0,3.0,0.0
8196,Zimbabwe,ZWE,2017-01-01,Presidential,30.0,1.0,1.0,3.0,1.0,1.0,...,4.0,0.0,0.0,0.0,0.0,30.0,30.0,4.0,4.0,0.0
8197,Zimbabwe,ZWE,2018-01-01,Presidential,1.0,1.0,0.0,3.0,1.0,1.0,...,4.0,0.25,0.25,0.333333,0.333333,5.0,5.0,1.0,1.0,0.0
8198,Zimbabwe,ZWE,2019-01-01,Presidential,2.0,1.0,4.0,3.0,1.0,1.0,...,4.0,0.0,0.0,0.0,0.0,6.0,6.0,2.0,2.0,0.0


## Create identifier to map countries in "Protests" to "Instituions"

In [4]:
# Import "Protests" dataset
engine = create_engine('sqlite:///../data/processed/protests.db')
with engine.begin() as connection:
    protests = pd.read_sql('SELECT * FROM protests', connection)

In [5]:
# Create dictionary to map different country names
protest_countries = pd.Series(protests.country.unique())
institution_countries = institutions.countryname.unique()

In [6]:
# Identify cases where a protest_country isn't in the institution_country
no_match = {}
for country in protest_countries:
    if country not in institution_countries:
#        print (country)
        no_match[country] = None
no_match

{'Dominican Republic': None,
 'United Kingdom': None,
 'Germany': None,
 'Germany West': None,
 'Germany East': None,
 'Czechoslovakia': None,
 'Czech Republic': None,
 'Slovak Republic': None,
 'Kosovo': None,
 'Serbia': None,
 'Bosnia': None,
 'Serbia and Montenegro': None,
 'Montenegro': None,
 'USSR': None,
 'Cape Verde': None,
 'Equatorial Guinea': None,
 'Ivory Coast': None,
 'Central African Republic': None,
 'Congo Brazzaville': None,
 'Congo Kinshasa': None,
 'South Africa': None,
 'Comoros': None,
 'United Arab Emirate': None,
 'China': None,
 'North Korea': None,
 'South Korea': None,
 'Timor Leste': None,
 'Papua New Guinea': None}

##### Given the above list, manually create a dictionary to find the "Institution" country name corresponding to the "Protest" country name, since it is likely a difference in syntax/spelling/etc

In [8]:
print(institution_countries)

['Turk Cyprus' 'Afghanistan' 'Angola' 'Albania' 'UAE' 'Argentina'
 'Armenia' 'Australia' 'Austria' 'Azerbaijan' 'Burundi' 'Belgium' 'Benin'
 'Burkina Faso' 'Bangladesh' 'Bulgaria' 'Bahrain' 'Bahamas' 'Bosnia-Herz'
 'Belarus' 'Belize' 'Bolivia' 'Brazil' 'Barbados' 'Brunei' 'Bhutan'
 'Botswana' 'Cent. Af. Rep.' 'Canada' 'Switzerland' 'Chile' 'PRC'
 "Cote d'Ivoire" 'Cameroon' 'Congo' 'Colombia' 'Comoro Is.' 'C. Verde Is.'
 'Costa Rica' 'Czech Rep.' 'Cuba' 'Cyprus' 'GDR' 'FRG/Germany' 'Djibouti'
 'Denmark' 'Dom. Rep.' 'Algeria' 'Ecuador' 'Egypt' 'Eritrea' 'Spain'
 'Estonia' 'Ethiopia' 'Finland' 'Fiji' 'France' 'Gabon' 'UK' 'Georgia'
 'Ghana' 'Guinea' 'Gambia' 'Guinea-Bissau' 'Eq. Guinea' 'Greece' 'Grenada'
 'Guatemala' 'Guyana' 'Honduras' 'Croatia' 'Haiti' 'Hungary' 'Indonesia'
 'India' 'Ireland' 'Iran' 'Iraq' 'Iceland' 'Israel' 'Italy' 'Jamaica'
 'Jordan' 'Japan' 'Kazakhstan' 'Kenya' 'Kyrgyzstan' 'Cambodia' 'ROK'
 'Kuwait' 'Laos' 'Lebanon' 'Liberia' 'Libya' 'St. Lucia' 'Sri Lanka'
 'Lesot

In [27]:
no_match['Dominican Republic'] = 'Dom. Rep.'
no_match['United Kingdom'] = 'UK'
no_match['Germany'] = 'FRG/Germany'
no_match['Germany West'] = 'FRG/Germany'
no_match['Germany East'] = 'GDR'
no_match['Czechoslovakia'] = 'Czech Rep.'
no_match['Czech Republic'] = 'Czech Rep.'
no_match['Slovak Republic'] = 'Slovakia'
no_match['Kosovo'] = None # No corresponding country in "Institutions" dataset
no_match['Serbia'] = None # No corresponding country in "Institutions" dataset
no_match['Bosnia'] = None # No corresponding country in "Institutions" dataset
no_match['Serbia and Montenegro'] = None # No corresponding country in "Institutions" dataset
no_match['Montenegro'] = None # No corresponding country in "Institutions" dataset
no_match['USSR'] = 'Soviet Union'
no_match['Cape Verde'] = 'C. Verde Is.'
no_match['Equatorial Guinea'] = 'Eq. Guinea'
no_match['Ivory Coast'] = "Cote d'Ivoire"
no_match['Central African Republic'] = 'Cent. Af. Rep.'
no_match['Congo Brazzaville'] = 'Congo'
no_match['Congo Kinshasa'] = 'Congo (DRC)'
no_match['South Africa'] = 'S. Africa'
no_match['Comoros'] = 'Comoro Is.'
no_match['United Arab Emirate'] = 'UAE'
no_match['China'] = 'PRC'
no_match['North Korea'] = 'PRK'
no_match['South Korea'] = 'ROK'
no_match['Timor Leste'] = 'Timor-Leste'
no_match['Papua New Guinea'] = 'P. N. Guinea'


keys = no_match.keys()
for country in list(no_match):
    if no_match[country] == None:
        del no_match[country]
        
no_match

{'Dominican Republic': 'Dom. Rep.',
 'United Kingdom': 'UK',
 'Germany West': 'FRG/Germany',
 'Germany East': 'GDR',
 'Czechoslovakia': 'Czech Rep.',
 'Czech Republic': 'Czech Rep.',
 'Slovak Republic': 'Slovakia',
 'USSR': 'Soviet Union',
 'Cape Verde': 'C. Verde Is.',
 'Equatorial Guinea': 'Eq. Guinea',
 'Ivory Coast': "Cote d'Ivoire",
 'Central African Republic': 'Cent. Af. Rep.',
 'Congo Brazzaville': 'Congo',
 'Congo Kinshasa': 'Congo (DRC)',
 'South Africa': 'S. Africa',
 'Comoros': 'Comoro Is.',
 'United Arab Emirate': 'UAE',
 'China': 'PRC',
 'North Korea': 'PRK',
 'South Korea': 'ROK',
 'Timor Leste': 'Timor-Leste',
 'Papua New Guinea': 'P. N. Guinea',
 'Germany': 'FRG/Germany'}

In [ ]:
# Create series for appending that shows "protest" formatting of country
country_name_converter = {}

for country in institution_countries:
    if country in no_match.

In [23]:
no_match

{'Dominican Republic': 'Dom. Rep.',
 'United Kingdom': 'UK',
 'Germany West': 'FRG/Germany',
 'Germany East': 'GDR',
 'Czechoslovakia': 'Czech Rep.',
 'Czech Republic': 'Czech Rep.',
 'Slovak Republic': 'Slovakia',
 'Kosovo': None,
 'Serbia': None,
 'Bosnia': None,
 'Serbia and Montenegro': None,
 'Montenegro': None,
 'USSR': 'Soviet Union',
 'Cape Verde': 'C. Verde Is.',
 'Equatorial Guinea': 'Eq. Guinea',
 'Ivory Coast': "Cote d'Ivoire",
 'Central African Republic': 'Cent. Af. Rep.',
 'Congo Brazzaville': 'Congo',
 'Congo Kinshasa': 'Congo (DRC)',
 'South Africa': 'S. Africa',
 'Comoros': 'Comoro Is.',
 'United Arab Emirate': 'UAE',
 'China': 'PRC',
 'North Korea': 'PRK',
 'South Korea': 'ROK',
 'Timor Leste': 'Timor-Leste',
 'Papua New Guinea': 'P. N. Guinea'}

In [14]:
# Create new dict, swapping values for keys and dropping "None"s
no_match_flip = {}
for country in no_match:
    val = no_match[country]
    if val  != None:
        # Swap value for key
        no_match_flip[val] = country

In [15]:
no_match_flip

{'Dom. Rep.': 'Dominican Republic',
 'UK': 'United Kingdom',
 'FRG/Germany': 'Germany West',
 'GDR': 'Germany East',
 'Czech Rep.': 'Czech Republic',
 'Slovakia': 'Slovak Republic',
 'Soviet Union': 'USSR',
 'C. Verde Is.': 'Cape Verde',
 'Eq. Guinea': 'Equatorial Guinea',
 "Cote d'Ivoire": 'Ivory Coast',
 'Cent. Af. Rep.': 'Central African Republic',
 'Congo': 'Congo Brazzaville',
 'Congo (DRC)': 'Congo Kinshasa',
 'S. Africa': 'South Africa',
 'Comoro Is.': 'Comoros',
 'UAE': 'United Arab Emirate',
 'PRC': 'China',
 'PRK': 'North Korea',
 'ROK': 'South Korea',
 'Timor-Leste': 'Timor Leste',
 'P. N. Guinea': 'Papua New Guinea'}

In [20]:
institutions.loc[institutions.countryname=='FRG/Germany'].sort_values(by='year')

,countryname,ifs,year,system,yrsoffc,finittrm,yrcurnt,termlimit,reelect,multpl,...,checks,stabs_strict,stabs,stabns_strict,stabns,tenlong_strict,tenlong,tenshort_strict,tenshort,polariz
1978,FRG/Germany,DEU,1975-01-01,Parliamentary,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN
1979,FRG/Germany,DEU,1976-01-01,Parliamentary,2.0,1.0,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN
1980,FRG/Germany,DEU,1977-01-01,Parliamentary,3.0,1.0,3.0,1.0,1.0,1.0,...,5.0,NaN,NaN,NaN,NaN,3.0,3.0,1.0,1.0,2.0
1981,FRG/Germany,DEU,1978-01-01,Parliamentary,4.0,1.0,2.0,1.0,1.0,1.0,...,5.0,0.0,0.0,0.0,0.0,4.0,4.0,2.0,2.0,2.0
1982,FRG/Germany,DEU,1979-01-01,Parliamentary,5.0,1.0,1.0,1.0,1.0,1.0,...,5.0,0.0,0.0,0.0,0.0,5.0,5.0,3.0,3.0,2.0
1983,FRG/Germany,DEU,1980-01-01,Parliamentary,6.0,1.0,0.0,1.0,1.0,1.0,...,5.0,0.0,0.0,0.0,0.0,6.0,6.0,4.0,4.0,2.0
1984,FRG/Germany,DEU,1981-01-01,Parliamentary,7.0,1.0,3.0,1.0,1.0,1.0,...,5.0,0.0,0.0,0.0,0.0,7.0,7.0,5.0,5.0,2.0
1985,FRG/Germany,DEU,1982-01-01,Parliamentary,8.0,1.0,2.0,1.0,1.0,1.0,...,5.0,0.0,0.0,0.0,0.0,8.0,8.0,6.0,6.0,2.0
1986,FRG/Germany,DEU,1983-01-01,Parliamentary,1.0,1.0,0.0,1.0,1.0,1.0,...,5.0,0.5,0.5,0.666667,0.666667,7.0,7.0,1.0,1.0,2.0
1987,FRG/Germany,DEU,1984-01-01,Parliamentary,2.0,1.0,3.0,1.0,1.0,1.0,...,5.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,2.0,2.0


### Type correction: year

In [ ]:
if type(institutions['year']) == type('hereisastring'):
    institutions['year'] = [int(x[:4]) for x in institutions['year']]

# Notes

- Look up dummy dataset from sklearn for "dumb" model
- 

# Export data to SQL 

In [ ]:
engine = create_engine('sqlite:///../data/processed/institutions.db')

with engine.begin() as connection:
    institutions.to_sql(name='institutions', con=connection, if_exists='replace')